In [1]:
import git
import pickle
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii

In [2]:
def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
    sharedECCKey = privKey * ciphertextPubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

In [18]:
privKey = secrets.randbelow(curve.field.n) #b
pubKeyB = privKey * curve.g #B

with open('pub_keyTEST.pickle', 'wb') as f:
    pickle.dump(pubKeyB, f)

In [53]:
with open('encrypt_data.pickle', 'rb') as f:
    encryptedMsg = pickle.load(f) #B

In [55]:
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
}
decryptedMsg = decrypt_ECC(encryptedMsg, privKey)
print("decrypted msg:", decryptedMsg.decode('UTF-8'))

decrypted msg: 26 °C



In [6]:

repo = git.Repo("/Users/Chris-OSX/Desktop/TEL252_Central_de_datos")
# pull down 
repo.remotes.origin.pull()

In [8]:
repo.remotes.origin.pull()

In [20]:
file = open('pub_keyTEST.pickle', 'rb')

<git.Remote "pub_keyTEST">

In [34]:
#repo.remotes.origin.add(repo,"pub_keyTEST","/Users/Chris-OSX/Desktop/TEL252_Central_de_datos")
repo.git.add(repo.working_dir)
commit_output = repo.git.commit(m="Commit msg")
push_output = repo.git.push('--set-upstream', repo.remote().name, "main")

"Branch 'main' set up to track remote branch 'main' from 'origin'."